In [113]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, AutoConfig
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
import numpy as np

In [114]:
# Load pretrained model and tokenizer
MODEL_NAME = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [115]:
# Load and preprocess dataset imdb
dataset = load_dataset("imdb")
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

In [116]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = np.mean(predictions == labels)
    return {"accuracy": accuracy}

In [117]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)
print("Tokenized dataset size:")
print("Train size:", len(tokenized_datasets["train"]))
print("Test size:", len(tokenized_datasets["test"]))
# Limit dataset sizes
DATA_SET_SIZE_TRAIN = 10000
DATA_SET_SIZE_TEST = 10000
tokenized_datasets["train"] = tokenized_datasets["train"].select(range(DATA_SET_SIZE_TRAIN))
tokenized_datasets["test"] = tokenized_datasets["test"].select(range(DATA_SET_SIZE_TEST))
print("Limited Tokenized dataset size:")
print("Train size:", len(tokenized_datasets["train"]))
print("Test size:", len(tokenized_datasets["test"]))

Tokenized dataset size:
Train size: 25000
Test size: 25000
Limited Tokenized dataset size:
Train size: 10000
Test size: 10000


In [118]:
# Evaluate the pretrained model
trainer = Trainer(model=model, compute_metrics=compute_metrics)
eval_results = trainer.evaluate(tokenized_datasets["test"])
print("Pretrained Model Accuracy:", eval_results)

Pretrained Model Accuracy: {'eval_loss': 0.6843681931495667, 'eval_model_preparation_time': 0.0, 'eval_accuracy': 0.7309, 'eval_runtime': 21.181, 'eval_samples_per_second': 472.12, 'eval_steps_per_second': 59.015}


In [119]:
#for name, module in model.named_modules():
#    print(name)

In [120]:
# Create and apply PEFT config
peft_config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    lora_dropout=0.1, 
    target_modules=["q_lin", "k_lin", "v_lin"]
)
peft_model = get_peft_model(model, peft_config)

In [121]:
# Training the PEFT model
training_args = TrainingArguments(
    output_dir="./peft_model_output",  # Output directory for saving the model
    num_train_epochs=2,                # <======
    per_device_train_batch_size=8,     # Adjust batch size
    per_device_eval_batch_size=8,      # Adjust evaluation batch size
    eval_strategy="epoch",             # Evaluate at the end of each epoch
    save_strategy="epoch",             # Save model at the end of each epoch
    logging_dir="./logs",              # Directory for logs
    logging_steps=10                   # Log every 10 steps
)

trainer_peft = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics
)

In [122]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# To make sure the model uses the GPU:
trainer_peft.model.to(device)

Using device: cuda


PeftModel(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): DistilBertSdpaAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): lora.Linear(
                  (base_layer): Linear(in_features=768, out_features=768, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=8, bias=F

In [123]:
# Train the PEFT model
trainer_peft.train()

Epoch,Training Loss,Validation Loss
1,0.009000,No log
2,0.007900,No log


TrainOutput(global_step=2500, training_loss=0.03174898165762424, metrics={'train_runtime': 148.5623, 'train_samples_per_second': 134.624, 'train_steps_per_second': 16.828, 'total_flos': 2662937518080000.0, 'train_loss': 0.03174898165762424, 'epoch': 2.0})

In [124]:
# Save the fine-tuned PEFT model
peft_model.save_pretrained("./peft_model_output")

In [125]:
# Load the saved PEFT model for inference
peft_model_loaded = AutoModelForSequenceClassification.from_pretrained("./peft_model_output")
trainer_peft_loaded = Trainer(
    model=peft_model_loaded,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [126]:
# Evaluate the fine-tuned PEFT model
eval_results_peft = trainer_peft_loaded.evaluate(tokenized_datasets["test"])

In [127]:
# Compare the fine-tuned PEFT model to the original pretrained model
print("Base Model Accuracy:", eval_results_base)
print("Fine-Tuned PEFT Model Accuracy:", eval_results_peft)

Base Model Accuracy: {'eval_loss': 0.6843681931495667, 'eval_model_preparation_time': 0.001, 'eval_accuracy': 0.7309, 'eval_runtime': 21.5145, 'eval_samples_per_second': 464.803, 'eval_steps_per_second': 58.1}
Fine-Tuned PEFT Model Accuracy: {'eval_loss': 0.5503780245780945, 'eval_model_preparation_time': 0.0021, 'eval_accuracy': 1.0, 'eval_runtime': 22.7575, 'eval_samples_per_second': 439.415, 'eval_steps_per_second': 54.927}


In [128]:
print("Original Text:", tokenized_datasets["train"][0]["text"]) 

Original Text: I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and f